In [132]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = '3ac843f881d943ff8cb08a2f660fc58e'
client_secret = 'e409995d02ba4c038fcd22b4b10c4ca6'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

import pandas as pd 
import time 

In [133]:
import requests
import datetime
from urllib.parse import urlencode
import base64

In [134]:
class SpotifyAPI(object):    
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [135]:
spotify = SpotifyAPI(client_id, client_secret)

In [136]:
import pandas as pd
df_bands = pd.read_csv('cincy_music_band_list.csv')
df_bands.columns = ['order', 'names', 'genres', 'favorites']
df_bands.head()

,order,names,genres,favorites
0,0,Trauma Illinois,rock\nalternative,70.0
1,1,PUBLIC,pop\nindie\nrock,90.0
2,2,The Tillers,acoustic\nfolk\nbluegrass,278.0
3,3,Cincinnati Symphony Orchestra,classical,372.0
4,4,The National,alternative\nrock\nindie,172.0


In [138]:
df_bands.sort_values('favorites',ascending=True, na_position='last').head(40)

,order,names,genres,favorites
351,351,Euxine,NaN,1.0
323,323,Justin Lynch& The New Usuals,folk\nsinger/songwriter\ncountry\nalternative folk,1.0
322,322,Moped mafia 1,punk\nrock,1.0
321,321,Queen City Cabaret,hip hop\nelectronic\nalternative,1.0
320,320,Radical Manifesto,rock\npunk,1.0
75,75,Hare Hunter Field,rock\nfolk\nblues,1.0
234,234,Bershy,rock\nindie rock\npsych rock\nhard rock,1.0
233,233,Fam Band,americana/bluegrass/folk,1.0
232,232,Sunderman & Sorentino,folk\namericana\nsinger/songwriter,1.0
231,231,Ashley Lilly,folk\nrock\njam\nroots\namericana\nnewgrass bluegrass americana\nbluegrass,1.0


In [90]:
# df_add_bands = pd.ExcelFile("add_cincy_bands.xlsx")\

# df_add_bands=pd.read_excel(
#      os.path.join("add_cincy_bands.xlsx"),
#      engine='openpyxl',
# )

df_add_bands = pd.read_csv('add_cincy_bands.csv')

df_add_bands.head(5)

,BANDS
0,The Grove
1,The Midwestern
2,Sundae Drives
3,500 Miles to Memphis
4,See You In The Funnies


In [91]:
df_scraped_ids = pd.read_csv('ids_scraped.csv')
df_scraped_ids.head(5)

,Unnamed: 0,spotify,id
0,0,https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo,GT8mZHoTsRGMkmownzCeo
1,1,https://open.spotify.com/artist/2VfhnjHV7e3M499VPa1w71?si=3AWGRElWQxCB401tYycWaA,VfhnjHV7e3M499VPa1w71?
2,2,https://open.spotify.com/artist/17ON1uqBgUZVPOB6suCTxM,7ON1uqBgUZVPOB6suCTxM
3,3,https://open.spotify.com/artist/77VTTs4IOPMNUiClgBg27D,7VTTs4IOPMNUiClgBg27D
4,4,https://open.spotify.com/artist/1AqoxUC6J6PdRjbc3wLlhk?si=6H50pCvlSk-o3w8_QgJv9w,AqoxUC6J6PdRjbc3wLlhk?


In [92]:
search = spotify.search(query="Trauma Illinois", search_type="artist")

q=Trauma+Illinois&type=artist


In [93]:
spotify.search({"artist": "7GT8mZHoTsRGMkmownzCeo"}, search_type="artist")

q=artist%3A7GT8mZHoTsRGMkmownzCeo&type=artist


{'artists': {'href': 'https://api.spotify.com/v1/search?query=artist%3A7GT8mZHoTsRGMkmownzCeo&type=artist&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [94]:
# search = spotify.search(query="7GT8mZHoTsRGMkmownzCeo", search_type="id")
# # get_artist('7GT8mZHoTsRGMkmownzCeo')
search = spotify.search(query="7GT8mZHoTsRGMkmownzCeo", search_type="artist")

q=7GT8mZHoTsRGMkmownzCeo&type=artist


In [95]:
# id = '7GT8mZHoTsRGMkmownzCeo'
# https://api.spotify.com/v1/artists/{id}/albums

In [96]:
pd.DataFrame(search.items())

,0,1
0,artists,"{'href': 'https://api.spotify.com/v1/search?query=7GT8mZHoTsRGMkmownzCeo&type=artist&offset=0&limit=20', 'items': [], 'limit': 20, 'next': None, 'offset': 0, 'previous': None, 'total': 0}"


In [97]:
search

{'artists': {'href': 'https://api.spotify.com/v1/search?query=7GT8mZHoTsRGMkmownzCeo&type=artist&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [139]:
df_bands = df_bands[['names','favorites']]
df_bands.head()

,names,favorites
0,Trauma Illinois,70.0
1,PUBLIC,90.0
2,The Tillers,278.0
3,Cincinnati Symphony Orchestra,372.0
4,The National,172.0


In [140]:
df_add_bands = df_add_bands.rename(columns={'BANDS': 'names'})
df_merge_bands = pd.concat([df_bands,df_add_bands], axis=0)
df_merge_bands.head(5)

,names,favorites
0,Trauma Illinois,70.0
1,PUBLIC,90.0
2,The Tillers,278.0
3,Cincinnati Symphony Orchestra,372.0
4,The National,172.0


In [141]:
len(df_merge_bands)

523

In [142]:
len(df_bands)

424

In [143]:
# spotify.search(query="Fam Band", search_type="artist")

df_spotify_info = pd.DataFrame(columns=["artist"])

# for row in df_bands.names():
#     name = row
#     spotify.search(query=name, search_type="artist")

for index, row in df_merge_bands.iterrows():
    print(row['names'])
    append = spotify.search(query=(row['names']), search_type="artist")
    df_spotify_info = df_spotify_info.append({"artist":append}, ignore_index=True)

# for band in band_test:
#     print(band.text)
#     df_likes = df_spotify.append({"spotify":band.text}, ignore_index=True)

wbees
q=The+Newbees&type=artist
Carriers
q=Carriers&type=artist
Frankly Speaking
q=Frankly+Speaking&type=artist
Telehope
q=Telehope&type=artist
Counterfeit Money Machine
q=Counterfeit+Money+Machine&type=artist
Jamwave
q=Jamwave&type=artist
Johnny Fink and The Intrusion
q=Johnny+Fink+and+The+Intrusion&type=artist
Kevin Stokley
q=Kevin+Stokley&type=artist
Brian Sauer
q=Brian+Sauer&type=artist
The Proper Authorities
q=The+Proper+Authorities&type=artist
Hare Hunter Field
q=Hare+Hunter+Field&type=artist
Ric Hickey
q=Ric+Hickey&type=artist
Jeremy Pinnell
q=Jeremy+Pinnell&type=artist
Audley
q=Audley&type=artist
Lauren Eylise
q=Lauren+Eylise&type=artist
Mike Oberst of The Tillers
q=Mike+Oberst+of+The+Tillers&type=artist
In The Pines
q=In+The+Pines&type=artist
Crooked Rook
q=Crooked+Rook&type=artist
Shiny Old Soul
q=Shiny+Old+Soul&type=artist
Ma Crow & Co.
q=Ma+Crow+%26+Co.&type=artist
Nicholas Clay
q=Nicholas+Clay&type=artist
The Vims
q=The+Vims&type=artist
Charlie John
q=Charlie+John&type=art

In [144]:
pd.set_option('max_colwidth', -1)
df_spotify_info.head(1)

,artist
0,"{'artists': {'href': 'https://api.spotify.com/v1/search?query=Trauma+Illinois&type=artist&offset=0&limit=20', 'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'}, 'followers': {'href': None, 'total': 524}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo', 'id': '7GT8mZHoTsRGMkmownzCeo', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/6990911ce4283e2c2025ad9889542a9a98272789', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/b43c21f903c8e81dac4c84f0d74b8ece139cf759', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ec04bda28f9d657d3358fc6472b6e4b43058597a', 'width': 160}], 'name': 'Trauma Illinois', 'popularity': 4, 'type': 'artist', 'uri': 'spotify:artist:7GT8mZHoTsRGMkmownzCeo'}], 'limit': 20, 'next': None, 'offset': 0, 'previous': None, 'total': 1}}"


In [145]:
# df_spotify_info['artist'].apply(pd.Series)
test = pd.json_normalize(df_spotify_info['artist'])
# pd.json_normalize(test['artists.items'])
test = test['artists.items'].apply(pd.Series)
test.columns = ['Main', 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
test.columns
test
trim_spotify = test['Main'].apply(pd.Series)
trim_spotify.head(2)

,0,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,NaN,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},"{'href': None, 'total': 524}",[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,"[{'height': 640, 'url': 'https://i.scdn.co/image/6990911ce4283e2c2025ad9889542a9a98272789', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/b43c21f903c8e81dac4c84f0d74b8ece139cf759', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ec04bda28f9d657d3358fc6472b6e4b43058597a', 'width': 160}]",Trauma Illinois,4.0,artist,spotify:artist:7GT8mZHoTsRGMkmownzCeo
1,NaN,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},"{'href': None, 'total': 148156}",[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,"[{'height': 640, 'url': 'https://i.scdn.co/image/1260f5a3496e92bb4a26c6dfae3ac2d08e325990', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/aaaf41e0952d7ed2e67f959abf8d0702ad5cc065', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/18c2da871321db86b93e11738e1dac583d78415c', 'width': 160}]",PUBLIC,69.0,artist,spotify:artist:4vxaQs6vK54nK89J1VtLex


In [146]:
trim_spotify['followers'] = trim_spotify['followers'].astype(str)
trim_spotify['popularity'] = trim_spotify['popularity'].astype(str)
trim_spotify.dtypes

0                float64
external_urls    object 
followers        object 
genres           object 
href             object 
id               object 
images           object 
name             object 
popularity       object 
type             object 
uri              object 
dtype: object

In [147]:
## Trim Dictionary to just values
trim_spotify['followers'] = trim_spotify['followers'].str[-4:]
trim_spotify['followers'] = trim_spotify['followers'].str[:3]
trim_spotify.head(2)

,0,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,NaN,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},524,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,"[{'height': 640, 'url': 'https://i.scdn.co/image/6990911ce4283e2c2025ad9889542a9a98272789', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/b43c21f903c8e81dac4c84f0d74b8ece139cf759', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ec04bda28f9d657d3358fc6472b6e4b43058597a', 'width': 160}]",Trauma Illinois,4.0,artist,spotify:artist:7GT8mZHoTsRGMkmownzCeo
1,NaN,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,"[{'height': 640, 'url': 'https://i.scdn.co/image/1260f5a3496e92bb4a26c6dfae3ac2d08e325990', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/aaaf41e0952d7ed2e67f959abf8d0702ad5cc065', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/18c2da871321db86b93e11738e1dac583d78415c', 'width': 160}]",PUBLIC,69.0,artist,spotify:artist:4vxaQs6vK54nK89J1VtLex


In [148]:
## Drop unnecessary columns
trim_spotify_clean = trim_spotify.drop([0, 'images','type','uri'], axis=1)
trim_spotify_clean.head(5)

,external_urls,followers,genres,href,id,name,popularity
0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},524,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0
1,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0
2,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},932,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,22.0
3,{'spotify': 'https://open.spotify.com/artist/47lbvBv6v9UShnf2jlHcmu'},592,[american orchestra],https://api.spotify.com/v1/artists/47lbvBv6v9UShnf2jlHcmu,47lbvBv6v9UShnf2jlHcmu,Cincinnati Symphony Orchestra,19.0
4,{'spotify': 'https://open.spotify.com/artist/2cCUtGK9sDU2EoElnk0GNB'},425,"[indie rock, modern rock, stomp and holler]",https://api.spotify.com/v1/artists/2cCUtGK9sDU2EoElnk0GNB,2cCUtGK9sDU2EoElnk0GNB,The National,74.0


In [149]:
# trim_spotify_clean.dtypes
# trim_spotify_clean["popularity"] = pd.to_numeric(trim_spotify_clean["popularity"])
trim_spotify_clean["popularity"] = trim_spotify_clean["popularity"].apply(pd.to_numeric, errors='coerce')
# trim_spotify_clean['order'] = trim_spotify_clean.Index
trim_spotify_local = trim_spotify_clean
trim_spotify_local.reset_index(inplace=True)
trim_spotify_local['order'] = trim_spotify_local['index']
trim_spotify_local

,index,external_urls,followers,genres,href,id,name,popularity,order
0,0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},524,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0,0
1,1,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,1
2,2,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},932,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,22.0,2
3,3,{'spotify': 'https://open.spotify.com/artist/47lbvBv6v9UShnf2jlHcmu'},592,[american orchestra],https://api.spotify.com/v1/artists/47lbvBv6v9UShnf2jlHcmu,47lbvBv6v9UShnf2jlHcmu,Cincinnati Symphony Orchestra,19.0,3
4,4,{'spotify': 'https://open.spotify.com/artist/2cCUtGK9sDU2EoElnk0GNB'},425,"[indie rock, modern rock, stomp and holler]",https://api.spotify.com/v1/artists/2cCUtGK9sDU2EoElnk0GNB,2cCUtGK9sDU2EoElnk0GNB,The National,74.0,4
...,...,...,...,...,...,...,...,...,...
518,518,{'spotify': 'https://open.spotify.com/artist/6TAV25x5AHhy4hbaBzr64c'},95,[],https://api.spotify.com/v1/artists/6TAV25x5AHhy4hbaBzr64c,6TAV25x5AHhy4hbaBzr64c,Ben Knight & the Welldiggers,1.0,518
519,519,{'spotify': 'https://open.spotify.com/artist/3xd1QDGm37Pblfqn5qR8P6'},54,[],https://api.spotify.com/v1/artists/3xd1QDGm37Pblfqn5qR8P6,3xd1QDGm37Pblfqn5qR8P6,Warrick & Lowell,0.0,519
520,520,{'spotify': 'https://open.spotify.com/artist/42ufGf70mJ6FERFQMBWauz'},: 2,[],https://api.spotify.com/v1/artists/42ufGf70mJ6FERFQMBWauz,42ufGf70mJ6FERFQMBWauz,Nicholas Clark,4.0,520
521,521,{'spotify': 'https://open.spotify.com/artist/2lELmBTpxqW7hYmgw64YgE'},61,[],https://api.spotify.com/v1/artists/2lELmBTpxqW7hYmgw64YgE,2lELmBTpxqW7hYmgw64YgE,Chelsea Ford & the Trouble,0.0,521


In [150]:
# import numpy as np
# trim_spotify_local['local'] = np.where((trim_spotify_local['order']>400) & (trim_spotify_local['popularity']>20) , 'no', 'yes')
# trim_to_local_only = trim_spotify_local[trim_spotify_local['local'] =='yes'] 
# trim_to_local_only

In [151]:
# # THIS IS A BAD RULE, GET RID OF THIS
# trim_spotify_local[trim_spotify_local['local'] =='no'] 

In [152]:
trim_to_local_only = trim_spotify_local
# Reassign to correct DF
trim_spotify_clean = trim_to_local_only

trim_spotify_clean = trim_spotify_clean.replace(': ','', regex=True)
trim_spotify_clean[['followers']] = trim_spotify_clean[['followers']].astype(float)

goodbye_national_list = ['Billie Eilish','Marc Anthony','Travis Barker','The National',
'Supertramp','A Day To Remember','Nancy Sinatra','Moira Dela Torre',
'Audrey Mika', 'BANKS','Vacations','Patsy Cline','Ricky Nelson',
'SOYEON of (G)I-DLE','Jameson Rodgers','RetroVision','Zooey Deschanel',
'Eurielle',	'Ashley Lilley','Rucci','Aaryan Shah','Hilary Hahn',
'The Original Broadway Cast Of School Of Rock','Alter Ego','Madleen Kane',
'Loren Hartman','Rarin','Ted Nugent','Wilderado','Buckethead','Ghost Rider',
'Celestial Aeon Project','Copper King','JUDGE','Greer','The Sweeplings','Focus','McEnroe','	Matt Fuze',
'Voodoo Glow Skulls','Louis Langrée','Here Come The Mummies','The Shivers','Sungazer','Ben Flocks','The Shivers','Night Owl',
'Blueprint','Drew Schueler','Kelly Thomas','Sundae Drives','Meridian','Dark Matters','Mitch De Klein','Mick Blankenship',
'Wayward Sons','Dan Campbell','Soften the Glare','Bad Veins','Continuados','The Internet','Matt Fuze','John Ford Coley',
'Johnny Christoff','FWD','Resonator Threads','Northbound','Flatline','JBroadway','Ruby Reese','The Juice Is Okay',
'The String Theory','American Sin','My Own Words','Harlott','Jesse Barnett','Ben Levin','Joesph Gillespie','Nosebleed',
'Patchworks','Orchards','The Spiders From Mars','Tim Snider','Livid','Dr. Head','Brian Kelly','',
'','','','','','','',''

]

pd.set_option('display.max_rows', 500)

trim_spotify_clean_er = trim_spotify_clean[~trim_spotify_clean['name'].isin(goodbye_national_list)]
trim_spotify_clean_er = trim_spotify_clean_er.drop_duplicates('name')
# trim_spotify_clean_er.sort_values('followers',ascending=False, na_position='last').head(40)
trim_spotify_clean_er.sort_values('popularity',ascending=False, na_position='last').head(40)
# trim_spotify_clean_er.sort_values('order',ascending=False, na_position='last').head(40)

,index,external_urls,followers,genres,href,id,name,popularity,order
1,1,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,1
447,447,{'spotify': 'https://open.spotify.com/artist/7lGQNJ5Ypof1wZrM0ysNIU'},685.0,[japanese chillhop],https://api.spotify.com/v1/artists/7lGQNJ5Ypof1wZrM0ysNIU,7lGQNJ5Ypof1wZrM0ysNIU,Celestial Alignment,58.0,447
305,305,{'spotify': 'https://open.spotify.com/artist/47SKKz21c5Zd3Fh8u8pMvP'},800.0,[otacore],https://api.spotify.com/v1/artists/47SKKz21c5Zd3Fh8u8pMvP,47SKKz21c5Zd3Fh8u8pMvP,Lizz Robinett,57.0,305
41,41,{'spotify': 'https://open.spotify.com/artist/0o9qzOHuHkGZ3xMFKpMFkC'},467.0,"[brostep, canadian electronic, complextro, dark clubbing, dubstep, edm, electro house, electro trash, electronic trap]",https://api.spotify.com/v1/artists/0o9qzOHuHkGZ3xMFKpMFkC,0o9qzOHuHkGZ3xMFKpMFkC,Black Tiger Sex Machine,53.0,41
503,503,{'spotify': 'https://open.spotify.com/artist/58bPSgeKpyyFlJ9LatULIO'},665.0,"[cincinnati indie, hopebeat, modern rock]",https://api.spotify.com/v1/artists/58bPSgeKpyyFlJ9LatULIO,58bPSgeKpyyFlJ9LatULIO,HARBOUR,52.0,503
180,180,{'spotify': 'https://open.spotify.com/artist/79S80ZWgVhIPMCHuvl6SkA'},837.0,"[alternative rock, art pop, art rock, dream pop, experimental, experimental rock, freak folk, industrial, industrial rock, no wave, noise rock, post-punk, post-rock]",https://api.spotify.com/v1/artists/79S80ZWgVhIPMCHuvl6SkA,79S80ZWgVhIPMCHuvl6SkA,Swans,51.0,180
12,12,{'spotify': 'https://open.spotify.com/artist/4bnCmwKU89Rt1zWnRIdSUa'},247.0,"[deep new americana, new americana, roots americana, southern americana]",https://api.spotify.com/v1/artists/4bnCmwKU89Rt1zWnRIdSUa,4bnCmwKU89Rt1zWnRIdSUa,Arlo McKinley,46.0,12
8,8,{'spotify': 'https://open.spotify.com/artist/5qRXoCC1CDTMTli9gplUvk'},1.0,"[cincinnati indie, piano rock]",https://api.spotify.com/v1/artists/5qRXoCC1CDTMTli9gplUvk,5qRXoCC1CDTMTli9gplUvk,Foxy Shazam,43.0,8
23,23,{'spotify': 'https://open.spotify.com/artist/6gwexMlbo78wmlBF8U2V6r'},703.0,[indie rockism],https://api.spotify.com/v1/artists/6gwexMlbo78wmlBF8U2V6r,6gwexMlbo78wmlBF8U2V6r,Jane Decker,42.0,23
22,22,{'spotify': 'https://open.spotify.com/artist/70fUpxdAr6t0LJw3xJmMhm'},125.0,"[cincinnati indie, stomp and holler]",https://api.spotify.com/v1/artists/70fUpxdAr6t0LJw3xJmMhm,70fUpxdAr6t0LJw3xJmMhm,Motherfolk,42.0,22


In [153]:
ids_only = trim_spotify_clean_er['id']
ids_only = [x for x in ids_only if str(x) != 'nan']
ids_only = list(set(ids_only))
ids_only

with open('list_ids.txt', 'w') as f:
    for item in ids_only:
        f.write("%s\n" % item)

In [154]:
df_bands.head()

,names,favorites
0,Trauma Illinois,70.0
1,PUBLIC,90.0
2,The Tillers,278.0
3,Cincinnati Symphony Orchestra,372.0
4,The National,172.0


In [155]:
df_cincy_spotify = pd.merge(df_bands,trim_spotify_clean_er, left_on='names',right_on='name')
df_cincy_spotify.head()

,names,favorites,index,external_urls,followers,genres,href,id,name,popularity,order
0,Trauma Illinois,70.0,0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},524.0,[],https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0,0
1,PUBLIC,90.0,1,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,1
2,The Tillers,278.0,2,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},932.0,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,22.0,2
3,Cincinnati Symphony Orchestra,372.0,3,{'spotify': 'https://open.spotify.com/artist/47lbvBv6v9UShnf2jlHcmu'},592.0,[american orchestra],https://api.spotify.com/v1/artists/47lbvBv6v9UShnf2jlHcmu,47lbvBv6v9UShnf2jlHcmu,Cincinnati Symphony Orchestra,19.0,3
4,Saving Escape,46.0,5,{'spotify': 'https://open.spotify.com/artist/2VfhnjHV7e3M499VPa1w71'},104.0,[],https://api.spotify.com/v1/artists/2VfhnjHV7e3M499VPa1w71,2VfhnjHV7e3M499VPa1w71,Saving Escape,21.0,5


In [156]:
df_bands.describe()

,favorites
count,352.000000
mean,36.687500
std,47.415411
min,1.000000
25%,4.750000
50%,19.000000
75%,53.000000
max,372.000000


In [157]:
df_cincy_spotify.dtypes

names            object 
favorites        float64
index            int64  
external_urls    object 
followers        float64
genres           object 
href             object 
id               object 
name             object 
popularity       float64
order            int64  
dtype: object

In [158]:
df_cincy_spotify['genres'] = df_cincy_spotify['genres'][1:]
df_cincy_spotify['genres'] = df_cincy_spotify['genres'][:-1]

## Clean Up Column Data


In [159]:
# df_cincy_spotify[['GENRE1','GENRE2','GENRE3']] = df_cincy_spotify['genres'].str.split(',', n=2, expand=True)
# df_cincy_spotify.head()

In [160]:
# df_cincy_spotify_genres

In [161]:
# #if one column DataFrame
# df_cincy_spotify_genres = pd.DataFrame(df_cincy_spotify['genres'].values.tolist(), index=df_cincy_spotify.index)

# #if multiple columns and need join new df to existing
# df_cincy_spotify = df_cincy_spotify.join(pd.DataFrame(df_cincy_spotify_genres.pop('genres').values.tolist(), index=df_cincy_spotify.index))

In [162]:
# df_cincy_spotify['GENRE1'] = df_cincy_spotify['genres'].str[0]
# df_cincy_spotify['GENRE2'] = df_cincy_spotify['genres'].str[0]
# df_cincy_spotify['GENRE3'] = df_cincy_spotify['genres'].str[0]

In [163]:
# df_cincy_spotify[['GENRE1','GENRE2','GENRE3']] = df_cincy_spotify['genres'].str.split(",",expand=True)
for i in range(len(df_cincy_spotify)):
    try:
        df_cincy_spotify['GENRE1'] = df_cincy_spotify['genres'].str[0]
    except:
        'NA'
for i in range(len(df_cincy_spotify)):
    try:
        df_cincy_spotify['GENRE2'] = df_cincy_spotify['genres'].str[1]
    except:
        'NA'
for i in range(len(df_cincy_spotify)):
    try:
        df_cincy_spotify['GENRE3'] = df_cincy_spotify['genres'].str[2]
    except:
        'NA'

df_cincy_spotify.head(5)

,names,favorites,index,external_urls,followers,genres,href,id,name,popularity,order,GENRE1,GENRE2,GENRE3
0,Trauma Illinois,70.0,0,{'spotify': 'https://open.spotify.com/artist/7GT8mZHoTsRGMkmownzCeo'},524.0,NaN,https://api.spotify.com/v1/artists/7GT8mZHoTsRGMkmownzCeo,7GT8mZHoTsRGMkmownzCeo,Trauma Illinois,4.0,0,NaN,NaN,NaN
1,PUBLIC,90.0,1,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,1,indie rockism,NaN,NaN
2,The Tillers,278.0,2,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},932.0,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,22.0,2,roots americana,NaN,NaN
3,Cincinnati Symphony Orchestra,372.0,3,{'spotify': 'https://open.spotify.com/artist/47lbvBv6v9UShnf2jlHcmu'},592.0,[american orchestra],https://api.spotify.com/v1/artists/47lbvBv6v9UShnf2jlHcmu,47lbvBv6v9UShnf2jlHcmu,Cincinnati Symphony Orchestra,19.0,3,american orchestra,NaN,NaN
4,Saving Escape,46.0,5,{'spotify': 'https://open.spotify.com/artist/2VfhnjHV7e3M499VPa1w71'},104.0,[],https://api.spotify.com/v1/artists/2VfhnjHV7e3M499VPa1w71,2VfhnjHV7e3M499VPa1w71,Saving Escape,21.0,5,NaN,NaN,NaN


In [164]:
df_cincy_spotify_clean = df_cincy_spotify
df_cincy_spotify_clean.dtypes

names            object 
favorites        float64
index            int64  
external_urls    object 
followers        float64
genres           object 
href             object 
id               object 
name             object 
popularity       float64
order            int64  
GENRE1           object 
GENRE2           object 
GENRE3           object 
dtype: object

In [165]:
df_cincy_spotify_clean['followers'] = df_cincy_spotify_clean['followers'].apply(pd.to_numeric, errors='coerce')
df_cincy_spotify_clean['popularity'] = df_cincy_spotify_clean['popularity'].apply(pd.to_numeric, errors='coerce')
df_cincy_spotify_clean.dtypes

names            object 
favorites        float64
index            int64  
external_urls    object 
followers        float64
genres           object 
href             object 
id               object 
name             object 
popularity       float64
order            int64  
GENRE1           object 
GENRE2           object 
GENRE3           object 
dtype: object

In [166]:
df_cincy_spotify_clean.sort_values('followers', ascending=False).head(10)

,names,favorites,index,external_urls,followers,genres,href,id,name,popularity,order,GENRE1,GENRE2,GENRE3
48,Carriers,62.0,66,{'spotify': 'https://open.spotify.com/artist/03Y6dXzmx4DFtBblTZpAw4'},995.0,[],https://api.spotify.com/v1/artists/03Y6dXzmx4DFtBblTZpAw4,03Y6dXzmx4DFtBblTZpAw4,Carriers,13.0,66,NaN,NaN,NaN
57,Lauren Eylise,54.0,79,{'spotify': 'https://open.spotify.com/artist/12dqbtoVLwnXTJfK2XpaJb'},979.0,[],https://api.spotify.com/v1/artists/12dqbtoVLwnXTJfK2XpaJb,12dqbtoVLwnXTJfK2XpaJb,Lauren Eylise,20.0,79,NaN,NaN,NaN
16,Cincinnati Pops Orchestra,253.0,20,{'spotify': 'https://open.spotify.com/artist/1FV3sKugQJFGWgWpMIA2Fy'},958.0,"[american orchestra, pops orchestra]",https://api.spotify.com/v1/artists/1FV3sKugQJFGWgWpMIA2Fy,1FV3sKugQJFGWgWpMIA2Fy,Cincinnati Pops Orchestra,34.0,20,american orchestra,pops orchestra,NaN
2,The Tillers,278.0,2,{'spotify': 'https://open.spotify.com/artist/5IYB6AHoUTP9zukPC2boFS'},932.0,[roots americana],https://api.spotify.com/v1/artists/5IYB6AHoUTP9zukPC2boFS,5IYB6AHoUTP9zukPC2boFS,The Tillers,22.0,2,roots americana,NaN,NaN
98,Softspoken,33.0,148,{'spotify': 'https://open.spotify.com/artist/3y8JSazi8hWvLiSFXQqGIk'},885.0,[american metalcore],https://api.spotify.com/v1/artists/3y8JSazi8hWvLiSFXQqGIk,3y8JSazi8hWvLiSFXQqGIk,Softspoken,33.0,148,american metalcore,NaN,NaN
120,Sylmar,24.0,194,{'spotify': 'https://open.spotify.com/artist/6ZyaO6lPHxQMapy52NBzHp'},812.0,[cincinnati indie],https://api.spotify.com/v1/artists/6ZyaO6lPHxQMapy52NBzHp,6ZyaO6lPHxQMapy52NBzHp,Sylmar,21.0,194,cincinnati indie,NaN,NaN
50,Telehope,47.0,68,{'spotify': 'https://open.spotify.com/artist/5cN9JUcmWKhbR3kM2mWIge'},802.0,"[cincinnati indie, hopebeat]",https://api.spotify.com/v1/artists/5cN9JUcmWKhbR3kM2mWIge,5cN9JUcmWKhbR3kM2mWIge,Telehope,34.0,68,cincinnati indie,hopebeat,NaN
88,Ghost Train,13.0,127,{'spotify': 'https://open.spotify.com/artist/7o9mgsmNx97zJa41WBDNqx'},785.0,[],https://api.spotify.com/v1/artists/7o9mgsmNx97zJa41WBDNqx,7o9mgsmNx97zJa41WBDNqx,Ghost Train,18.0,127,NaN,NaN,NaN
32,Over The Rhine,145.0,46,{'spotify': 'https://open.spotify.com/artist/1z26dqCz3PySR4G7HWsn93'},728.0,"[acoustic pop, ectofolk, folk]",https://api.spotify.com/v1/artists/1z26dqCz3PySR4G7HWsn93,1z26dqCz3PySR4G7HWsn93,Over The Rhine,39.0,46,acoustic pop,ectofolk,folk
167,K-Drama,10.0,282,{'spotify': 'https://open.spotify.com/artist/1PXBwNCd6LaX8iJTDwdC30'},710.0,"[christian hip hop, christian trap, gospel rap]",https://api.spotify.com/v1/artists/1PXBwNCd6LaX8iJTDwdC30,1PXBwNCd6LaX8iJTDwdC30,K-Drama,31.0,282,christian hip hop,christian trap,gospel rap


In [167]:
df_cincy_spotify_clean.sort_values('popularity', ascending=False).head(10)

,names,favorites,index,external_urls,followers,genres,href,id,name,popularity,order,GENRE1,GENRE2,GENRE3
1,PUBLIC,90.0,1,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,1,indie rockism,NaN,NaN
8,Arlo McKinley,119.0,12,{'spotify': 'https://open.spotify.com/artist/4bnCmwKU89Rt1zWnRIdSUa'},247.0,"[deep new americana, new americana, roots americana, southern americana]",https://api.spotify.com/v1/artists/4bnCmwKU89Rt1zWnRIdSUa,4bnCmwKU89Rt1zWnRIdSUa,Arlo McKinley,46.0,12,deep new americana,new americana,roots americana
6,Foxy Shazam,183.0,8,{'spotify': 'https://open.spotify.com/artist/5qRXoCC1CDTMTli9gplUvk'},1.0,"[cincinnati indie, piano rock]",https://api.spotify.com/v1/artists/5qRXoCC1CDTMTli9gplUvk,5qRXoCC1CDTMTli9gplUvk,Foxy Shazam,43.0,8,cincinnati indie,piano rock,NaN
19,Jane Decker,63.0,23,{'spotify': 'https://open.spotify.com/artist/6gwexMlbo78wmlBF8U2V6r'},703.0,[indie rockism],https://api.spotify.com/v1/artists/6gwexMlbo78wmlBF8U2V6r,6gwexMlbo78wmlBF8U2V6r,Jane Decker,42.0,23,indie rockism,NaN,NaN
18,Motherfolk,98.0,22,{'spotify': 'https://open.spotify.com/artist/70fUpxdAr6t0LJw3xJmMhm'},125.0,"[cincinnati indie, stomp and holler]",https://api.spotify.com/v1/artists/70fUpxdAr6t0LJw3xJmMhm,70fUpxdAr6t0LJw3xJmMhm,Motherfolk,42.0,22,cincinnati indie,stomp and holler,NaN
32,Over The Rhine,145.0,46,{'spotify': 'https://open.spotify.com/artist/1z26dqCz3PySR4G7HWsn93'},728.0,"[acoustic pop, ectofolk, folk]",https://api.spotify.com/v1/artists/1z26dqCz3PySR4G7HWsn93,1z26dqCz3PySR4G7HWsn93,Over The Rhine,39.0,46,acoustic pop,ectofolk,folk
154,Calumet,22.0,257,{'spotify': 'https://open.spotify.com/artist/0XIAEZvFmaxsJwhs5CvSAU'},147.0,[],https://api.spotify.com/v1/artists/0XIAEZvFmaxsJwhs5CvSAU,0XIAEZvFmaxsJwhs5CvSAU,Calumet,36.0,257,NaN,NaN,NaN
21,Coastal Club,81.0,27,{'spotify': 'https://open.spotify.com/artist/6Ssvxfw2kfG1jkfDJDGJoh'},25.0,[cincinnati indie],https://api.spotify.com/v1/artists/6Ssvxfw2kfG1jkfDJDGJoh,6Ssvxfw2kfG1jkfDJDGJoh,Coastal Club,35.0,27,cincinnati indie,NaN,NaN
50,Telehope,47.0,68,{'spotify': 'https://open.spotify.com/artist/5cN9JUcmWKhbR3kM2mWIge'},802.0,"[cincinnati indie, hopebeat]",https://api.spotify.com/v1/artists/5cN9JUcmWKhbR3kM2mWIge,5cN9JUcmWKhbR3kM2mWIge,Telehope,34.0,68,cincinnati indie,hopebeat,NaN
16,Cincinnati Pops Orchestra,253.0,20,{'spotify': 'https://open.spotify.com/artist/1FV3sKugQJFGWgWpMIA2Fy'},958.0,"[american orchestra, pops orchestra]",https://api.spotify.com/v1/artists/1FV3sKugQJFGWgWpMIA2Fy,1FV3sKugQJFGWgWpMIA2Fy,Cincinnati Pops Orchestra,34.0,20,american orchestra,pops orchestra,NaN


In [168]:
len(df_cincy_spotify_clean)

224

In [169]:
df_cincy_spotify_clean_no_dups = df_cincy_spotify_clean.loc[df_cincy_spotify_clean.astype(str).drop_duplicates('name').index]
len(df_cincy_spotify_clean_no_dups)

223

### Write Out Artist Info

In [170]:
df_cincy_spotify_clean_no_dups.to_csv("artists_info.csv", sep = ',')

In [171]:
df_cincy_spotify_clean_no_dups.sort_values('popularity',ascending=False, na_position='last')

,names,favorites,index,external_urls,followers,genres,href,id,name,popularity,order,GENRE1,GENRE2,GENRE3
1,PUBLIC,90.0,1,{'spotify': 'https://open.spotify.com/artist/4vxaQs6vK54nK89J1VtLex'},156.0,[indie rockism],https://api.spotify.com/v1/artists/4vxaQs6vK54nK89J1VtLex,4vxaQs6vK54nK89J1VtLex,PUBLIC,69.0,1,indie rockism,NaN,NaN
8,Arlo McKinley,119.0,12,{'spotify': 'https://open.spotify.com/artist/4bnCmwKU89Rt1zWnRIdSUa'},247.0,"[deep new americana, new americana, roots americana, southern americana]",https://api.spotify.com/v1/artists/4bnCmwKU89Rt1zWnRIdSUa,4bnCmwKU89Rt1zWnRIdSUa,Arlo McKinley,46.0,12,deep new americana,new americana,roots americana
6,Foxy Shazam,183.0,8,{'spotify': 'https://open.spotify.com/artist/5qRXoCC1CDTMTli9gplUvk'},1.0,"[cincinnati indie, piano rock]",https://api.spotify.com/v1/artists/5qRXoCC1CDTMTli9gplUvk,5qRXoCC1CDTMTli9gplUvk,Foxy Shazam,43.0,8,cincinnati indie,piano rock,NaN
19,Jane Decker,63.0,23,{'spotify': 'https://open.spotify.com/artist/6gwexMlbo78wmlBF8U2V6r'},703.0,[indie rockism],https://api.spotify.com/v1/artists/6gwexMlbo78wmlBF8U2V6r,6gwexMlbo78wmlBF8U2V6r,Jane Decker,42.0,23,indie rockism,NaN,NaN
18,Motherfolk,98.0,22,{'spotify': 'https://open.spotify.com/artist/70fUpxdAr6t0LJw3xJmMhm'},125.0,"[cincinnati indie, stomp and holler]",https://api.spotify.com/v1/artists/70fUpxdAr6t0LJw3xJmMhm,70fUpxdAr6t0LJw3xJmMhm,Motherfolk,42.0,22,cincinnati indie,stomp and holler,NaN
32,Over The Rhine,145.0,46,{'spotify': 'https://open.spotify.com/artist/1z26dqCz3PySR4G7HWsn93'},728.0,"[acoustic pop, ectofolk, folk]",https://api.spotify.com/v1/artists/1z26dqCz3PySR4G7HWsn93,1z26dqCz3PySR4G7HWsn93,Over The Rhine,39.0,46,acoustic pop,ectofolk,folk
154,Calumet,22.0,257,{'spotify': 'https://open.spotify.com/artist/0XIAEZvFmaxsJwhs5CvSAU'},147.0,[],https://api.spotify.com/v1/artists/0XIAEZvFmaxsJwhs5CvSAU,0XIAEZvFmaxsJwhs5CvSAU,Calumet,36.0,257,NaN,NaN,NaN
21,Coastal Club,81.0,27,{'spotify': 'https://open.spotify.com/artist/6Ssvxfw2kfG1jkfDJDGJoh'},25.0,[cincinnati indie],https://api.spotify.com/v1/artists/6Ssvxfw2kfG1jkfDJDGJoh,6Ssvxfw2kfG1jkfDJDGJoh,Coastal Club,35.0,27,cincinnati indie,NaN,NaN
16,Cincinnati Pops Orchestra,253.0,20,{'spotify': 'https://open.spotify.com/artist/1FV3sKugQJFGWgWpMIA2Fy'},958.0,"[american orchestra, pops orchestra]",https://api.spotify.com/v1/artists/1FV3sKugQJFGWgWpMIA2Fy,1FV3sKugQJFGWgWpMIA2Fy,Cincinnati Pops Orchestra,34.0,20,american orchestra,pops orchestra,NaN
50,Telehope,47.0,68,{'spotify': 'https://open.spotify.com/artist/5cN9JUcmWKhbR3kM2mWIge'},802.0,"[cincinnati indie, hopebeat]",https://api.spotify.com/v1/artists/5cN9JUcmWKhbR3kM2mWIge,5cN9JUcmWKhbR3kM2mWIge,Telehope,34.0,68,cincinnati indie,hopebeat,NaN
